In [1]:
import gurobipy as gp
import pandas as pd

In [2]:
distances = pd.read_csv('distances.csv')
pairs = pd.read_csv('pairs.csv')

## Question 1.2

In [31]:
#generating the possible blood type combinations
BT = [("A","A"),("A","AB"),("B","B"),("B","AB"),("AB","AB"),("O","A"),("O","B"),("O","AB"),("O","O")]

#creating the list of the potential matches - naming it C as in the assignment description
C = []

for i in range(len(pairs)):
    #defining the donor location
    donorLoc = pairs.iloc[i]["Location"]
    for j in range(len(pairs)):
        #defining the receiver location
        receiverLoc = pairs.iloc[j]["Location"]
        #check whether our pairs have compatible blood types
        if (((pairs.iloc[i]["DBT"],pairs.iloc[j]["RBT"]) in BT) 
            #check whether their age difference is less or equal to 10
            and (abs(int(pairs.iloc[i]["Dage"])-int(pairs.iloc[j]["Rage"])) <= 10)
            #check whether their distance is less than 300 miles or they are located in the same city
            and ((donorLoc == receiverLoc) or 
                 ((distances[((distances["city1"]==donorLoc) & 
                              (distances["city2"]==receiverLoc))]["distance"]) <= 300).any())):
            #store the potential match if it meets all our criteria and add it to the C list
            bloodmatch = (pairs.iloc[i]["ID"],pairs.iloc[j]["ID"])
            C.append(bloodmatch)

In [32]:
#converting the list to a data frame to make operations with it easier
dfC = pd.DataFrame(C)
#first column are the potential donors (including pairs)
potential_donors = set(dfC[0])
#second column are the potential receivers (including pairs)
potential_rec = set(dfC[1])

In [33]:
#next up is the creation of a list with all the observations, which are pairs in our potential matches list C - I name it P
P = list(pairs["ID"][pairs["type"]=="pair"])
#this just collects all the pairs that are on the donor side
a = set(P).intersection(potential_donors)
#this collects all the pairs on the receiver side
b = set(P).intersection(potential_rec)
#here we combine the pairs on the donor and receiver side into one temporary list
temp = []
temp.extend(a)
temp.extend(b)

In [34]:
#we collect all the unique ID's of pairs on both donor and receiver side and store it into a new list called potential_pairs
potential_pairs = set(temp)

## Question 1.3 - Gurobi Implementation

In [35]:
#C is defined above

model = gp.Model()
x = model.addVars(C, vtype = gp.GRB.BINARY, name = " ")
#objective is to maximize the number of transplants, which are given by the binary variable x[i,j]
model.setObjective(gp.quicksum(x[match] for match in C),gp.GRB.MAXIMIZE)
#a potential donor can only donate max 1 kidney - here we tell Gurobi to only consider potential pairs in the created list
#for which the donor is the i
model.addConstrs(gp.quicksum(x[i,j] for j in (dfC[dfC[0]==i][1])) <= 1 for i in (potential_donors))
#a potential receiver can only receive max 1 kidney
model.addConstrs(gp.quicksum(x[i,j] for i in (dfC[dfC[1]==j][0])) <= 1 for j in (potential_rec))
#a pair only donates a kidney only if they receive one and vice versa
model.addConstrs(gp.quicksum(x[i,j] for j in (dfC[dfC[0]==i][1])) == 
                 gp.quicksum(x[k,i] for k in (dfC[dfC[1]==i][0])) for i in potential_pairs)
model.optimize()
model.printAttr("X")


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 369 rows, 896 columns and 2932 nonzeros
Model fingerprint: 0x34755998
Variable types: 0 continuous, 896 integer (896 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 45.0000000
Presolve removed 192 rows and 251 columns
Presolve time: 0.02s
Presolved: 177 rows, 645 columns, 2057 nonzeros
Found heuristic solution: objective 75.0000000
Variable types: 0 continuous, 645 integer (645 binary)

Root relaxation: objective 9.000000e+01, 167 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      90.0000000   90.00000  0.00

In [36]:
#code to extract the optimal solution as a list
optSolution = [k for k, v in x.items() if v.X > 0] 
optSolution

[(4, 187),
 (8, 8),
 (10, 35),
 (14, 164),
 (19, 182),
 (20, 93),
 (29, 29),
 (30, 133),
 (31, 20),
 (32, 32),
 (33, 10),
 (34, 34),
 (35, 102),
 (36, 60),
 (40, 180),
 (41, 194),
 (42, 82),
 (45, 118),
 (46, 94),
 (54, 40),
 (55, 151),
 (58, 58),
 (60, 108),
 (61, 89),
 (63, 158),
 (64, 190),
 (67, 115),
 (68, 113),
 (73, 73),
 (81, 84),
 (82, 148),
 (84, 197),
 (85, 110),
 (87, 146),
 (89, 196),
 (91, 23),
 (92, 85),
 (93, 98),
 (94, 159),
 (95, 99),
 (97, 11),
 (98, 90),
 (99, 152),
 (102, 33),
 (103, 167),
 (104, 145),
 (105, 161),
 (108, 81),
 (110, 19),
 (111, 46),
 (113, 199),
 (114, 114),
 (115, 30),
 (116, 116),
 (117, 117),
 (119, 54),
 (120, 120),
 (121, 92),
 (122, 122),
 (123, 119),
 (131, 47),
 (133, 4),
 (136, 91),
 (138, 79),
 (143, 123),
 (144, 144),
 (145, 103),
 (147, 170),
 (148, 42),
 (152, 112),
 (153, 128),
 (155, 191),
 (157, 157),
 (159, 104),
 (161, 171),
 (164, 9),
 (167, 49),
 (169, 169),
 (171, 87),
 (172, 97),
 (175, 121),
 (176, 176),
 (180, 143),
 (186, 

## Question 1.4

In [37]:
#converting the solution list into a data frame
new_list_v2 = pd.DataFrame(optSolution)
new_list_v2

,0,1
0,4,187
1,8,8
2,10,35
3,14,164
4,19,182
...,...,...
85,193,138
86,194,70
87,196,65
88,197,38


### Question 1.4b

In [38]:
R = list(pairs["ID"][pairs["type"]=="receiver"])
D = list(pairs["ID"][pairs["type"]=="donor"])
P = list(pairs["ID"][pairs["type"]=="pair"])

optReceivers = list(new_list_v2[1])
unpairedrec = set(optReceivers).intersection(R)    
b1 = (len(unpairedrec)/len(R))
print(b1)

pairedrec = set(optReceivers).intersection(P)
b2 = ((len(pairedrec)/len(P)))
print(b2)

0.25
0.7291666666666666


### Question 1.4c

In [39]:
optDonors = list(new_list_v2[0])
unpaireddon = set(optDonors).intersection(D)
c = (len(unpaireddon)/len(D))
c

0.8333333333333334

### Question 1.4d

In [40]:
#finding the all the blood combinations of the original pairs 

originalpair = []
for i in range(len(pairs)):
    if pairs.iloc[i]["type"] == "pair":
        great = ((pairs.iloc[i]["DBT"],pairs.iloc[i]["RBT"]))
        originalpair.append(great)
    
originalpairdf = pd.DataFrame(originalpair)
                 
from collections import Counter    
Counter(originalpair)   

m = Counter({('B', 'A'): 6,
         ('A', 'O'): 21,
         ('O', 'A'): 17,
         ('O', 'O'): 23,
         ('A', 'B'): 4,
         ('O', 'B'): 3,
         ('B', 'O'): 8,
         ('AB', 'O'): 1,
         ('A', 'A'): 7,
         ('AB', 'A'): 2,
         ('AB', 'AB'): 2,
         ('A', 'AB'): 1,
         ('O', 'AB'): 1})

originalpaircount = pd.DataFrame.from_dict(m, orient='index').reset_index()
originalpaircount

,index,0
0,"(B, A)",6
1,"(A, O)",21
2,"(O, A)",17
3,"(O, O)",23
4,"(A, B)",4
5,"(O, B)",3
6,"(B, O)",8
7,"(AB, O)",1
8,"(A, A)",7
9,"(AB, A)",2


In [41]:
#these are the blood combinations for the pairs that received a kidney 

pairedrecdf= pd.DataFrame(pairedrec)
optPair = []

for i in range(len(pairedrecdf)):
    for j in range(len(pairs)):
        if pairedrecdf.iloc[i][0] == pairs.iloc[j]["ID"]:
            yessir = (pairs.iloc[j]["DBT"],pairs.iloc[j]["RBT"])
            optPair.append(yessir)
Counter(optPair)

p = Counter({('A', 'O'): 15,
         ('O', 'B'): 3,
         ('O', 'A'): 16,
         ('A', 'A'): 7,
         ('AB', 'AB'): 1,
         ('O', 'O'): 20,
         ('A', 'AB'): 1,
         ('B', 'A'): 2,
         ('A', 'B'): 2,
         ('O', 'AB'): 1,
         ('AB', 'A'): 1,
         ('B', 'O'): 1})
optPaircount = pd.DataFrame.from_dict(p, orient='index').reset_index()
optPaircount

,index,0
0,"(A, O)",15
1,"(O, B)",3
2,"(O, A)",16
3,"(A, A)",7
4,"(AB, AB)",1
5,"(O, O)",20
6,"(A, AB)",1
7,"(B, A)",2
8,"(A, B)",2
9,"(O, AB)",1


### Question 1.4e

In [42]:
#these are all the cities of the pairs from the original data

allcityv2 = []
for i in range(len(pairs)):
    if pairs.iloc[i]["type"] == "pair":
        allcity = ((pairs.iloc[i]["Location"]))
        allcityv2.append(allcity)
        
  
Counter(allcityv2)

o = Counter({'Moscow': 10,
         'Frankfurt': 6,
         'Vienna': 4,
         'Milan': 3,
         'Madrid': 3,
         'Düsseldorf': 7,
         'Lisbon': 5,
         'Amsterdam': 6,
         'Oslo': 4,
         'Zürich': 4,
         'Copenhagen': 7,
         'Istanbul': 9,
         'London': 6,
         'Palma de Mallorca': 6,
         'Manchester': 6,
         'Dublin': 5,
         'Rome': 3,
         'Paris': 2})
allcityCount = pd.DataFrame.from_dict(o, orient='index').reset_index()
allcityCount

,index,0
0,Moscow,10
1,Frankfurt,6
2,Vienna,4
3,Milan,3
4,Madrid,3
5,Düsseldorf,7
6,Lisbon,5
7,Amsterdam,6
8,Oslo,4
9,Zürich,4


In [43]:
#These are the cities of the pairs from the optimal solution

cityopt = []
for i in range(len(pairedrecdf)):
    for j in range(len(pairs)):
        if pairedrecdf.iloc[i][0] == pairs.iloc[j]["ID"]:
            locator = (pairs.iloc[j]["Location"])
            cityopt.append(locator)
Counter(cityopt)

from collections import Counter
d = Counter({'Frankfurt': 6,
         'Düsseldorf': 6,
         'Vienna': 2,
         'Milan': 3,
         'Istanbul': 8,
         'Amsterdam': 4,
         'Moscow': 5,
         'Oslo': 3,
         'Manchester': 6,
         'Lisbon': 5,
         'Zürich': 4,
         'Copenhagen': 4,
         'Dublin': 3,
         'Rome': 2,
         'London': 5,
         'Palma de Mallorca': 2,
         'Madrid': 1,
         'Paris': 1})
cityoptCount = pd.DataFrame.from_dict(d, orient='index').reset_index()
cityoptCount

,index,0
0,Frankfurt,6
1,Düsseldorf,6
2,Vienna,2
3,Milan,3
4,Istanbul,8
5,Amsterdam,4
6,Moscow,5
7,Oslo,3
8,Manchester,6
9,Lisbon,5


We copied all these tables into Excel to create the graphs and table in the document. 

## Question 1.5

The same code with the same variable names we ran. The only difference was the change in the constraint from less than and equal to 300 miles to less than and equal to 500 miles (below). Here all the code was commented it out to not be redundant, but if you want to run this code, we recommend to run it **after** running the code for question 2, otherwise question 2 will use the 500 miles solution rather than the 300 mile one. 

In [44]:
# BT = [("A","A"),("A","AB"),("B","B"),("B","AB"),("AB","AB"),("O","A"),("O","B"),("O","AB"),("O","O")]
# C = []

# for i in range(len(pairs)):
#     donorLoc = pairs.iloc[i]["Location"]
#     for j in range(len(pairs)):
#         receiverLoc = pairs.iloc[j]["Location"]
#         if (((pairs.iloc[i]["DBT"],pairs.iloc[j]["RBT"]) in BT) 
#             and (abs(int(pairs.iloc[i]["Dage"])-int(pairs.iloc[j]["Rage"])) <= 10)
#             and ((donorLoc == receiverLoc) or 
#                  ((distances[((distances["city1"]==donorLoc) & 
#                               (distances["city2"]==receiverLoc))]["distance"]) <= 500).any())):
#             bloodmatch = (pairs.iloc[i]["ID"],pairs.iloc[j]["ID"])
#             C.append(bloodmatch)

In [17]:
# dfC = pd.DataFrame(C)
# potential_donors = set(dfC[0])
# potential_rec = set(dfC[1])

In [18]:
# P = list(pairs["ID"][pairs["type"]=="pair"])
# a = set(P).intersection(potential_donors)
# b = set(P).intersection(potential_rec)
# temp = []
# temp.extend(a)
# temp.extend(b)

In [19]:
# potential_pairs = set(temp)

In [20]:
# #C is defined above

# model = gp.Model()
# x = model.addVars(C, vtype = gp.GRB.BINARY, name = " ")
# model.setObjective(gp.quicksum(x[match] for match in C),gp.GRB.MAXIMIZE)
# model.addConstrs(gp.quicksum(x[i,j] for j in (dfC[dfC[0]==i][1])) <= 1 for i in (potential_donors))
# model.addConstrs(gp.quicksum(x[i,j] for i in (dfC[dfC[1]==j][0])) <= 1 for j in (potential_rec))
# model.addConstrs(gp.quicksum(x[i,j] for j in (dfC[dfC[0]==i][1])) == gp.quicksum(x[k,i] for k in (dfC[dfC[1]==i][0])) for i in potential_pairs)
# model.optimize()
# model.printAttr("X")


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 381 rows, 1594 columns and 5231 nonzeros
Model fingerprint: 0x8785fd99
Variable types: 0 continuous, 1594 integer (1594 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 49.0000000
Presolve removed 151 rows and 267 columns
Presolve time: 0.02s
Presolved: 230 rows, 1327 columns, 4297 nonzeros
Found heuristic solution: objective 82.0000000
Variable types: 0 continuous, 1327 integer (1327 binary)

Root relaxation: objective 1.040000e+02, 239 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     104.0000000  104.00000

In [21]:
# optSolution = [k for k, v in x.items() if v.X > 0] 
# optSolution

# new_list_v2 = pd.DataFrame(optSolution)
# new_list_v2

,0,1
0,4,47
1,8,97
2,10,4
3,12,29
4,13,134
...,...,...
99,193,199
100,194,20
101,196,12
102,197,38


In [22]:
# R = list(pairs["ID"][pairs["type"]=="receiver"])
# D = list(pairs["ID"][pairs["type"]=="donor"])
# P = list(pairs["ID"][pairs["type"]=="pair"])

# optReceivers = list(new_list_v2[1])
# unpairedrec = set(optReceivers).intersection(R)    
# b1 = (len(unpairedrec)/len(R))
# b1

0.2875

In [23]:
# pairedrec = set(optReceivers).intersection(P)
# b2 = ((len(pairedrec)/len(P)))
# b2

0.84375

In [24]:
# optDonors = list(new_list_v2[0])
# unpaireddon = set(optDonors).intersection(D)
# c = (len(unpaireddon)/len(D))
# c

0.9583333333333334

#Question 5 answers

5. Analyze Solution For the optimal solution:
(a) How many transplantations will take place according to your solution? 
- 104

(b) What percentage of paired patients and what percentage of un-paired patients will
receive a kidney?
- paired patients = 0.2875
- unpaired patients = 0.84375

(c) What percentage of un-paired donors donate a kidney?
- 0.9583333333333334

In [25]:
# #finding the all the blood combinations of the original pairs 

# originalpair = []
# for i in range(len(pairs)):
#     if pairs.iloc[i]["type"] == "pair":
#         great = ((pairs.iloc[i]["DBT"],pairs.iloc[i]["RBT"]))
#         originalpair.append(great)
    
# originalpairdf = pd.DataFrame(originalpair)
                 
# from collections import Counter    
# Counter(originalpair)



# m = Counter({('B', 'A'): 6,
#          ('A', 'O'): 21,
#          ('O', 'A'): 17,
#          ('O', 'O'): 23,
#          ('A', 'B'): 4,
#          ('O', 'B'): 3,
#          ('B', 'O'): 8,
#          ('AB', 'O'): 1,
#          ('A', 'A'): 7,
#          ('AB', 'A'): 2,
#          ('AB', 'AB'): 2,
#          ('A', 'AB'): 1,
#          ('O', 'AB'): 1})

# originalpaircount = pd.DataFrame.from_dict(m, orient='index').reset_index()
# originalpaircount

,index,0
0,"(B, A)",6
1,"(A, O)",21
2,"(O, A)",17
3,"(O, O)",23
4,"(A, B)",4
5,"(O, B)",3
6,"(B, O)",8
7,"(AB, O)",1
8,"(A, A)",7
9,"(AB, A)",2


In [26]:

# pairedrecdf= pd.DataFrame(pairedrec)
# optPair = []

# for i in range(len(pairedrecdf)):
#     for j in range(len(pairs)):
#         if pairedrecdf.iloc[i][0] == pairs.iloc[j]["ID"]:
#             yessir = (pairs.iloc[j]["DBT"],pairs.iloc[j]["RBT"])
#             optPair.append(yessir)
# Counter(optPair)

# p = Counter({('A', 'O'): 18,
#          ('O', 'A'): 17,
#          ('O', 'O'): 22,
#          ('B', 'A'): 4,
#          ('A', 'B'): 3,
#          ('O', 'B'): 3,
#          ('B', 'O'): 3,
#          ('A', 'A'): 7,
#          ('AB', 'A'): 1,
#          ('AB', 'AB'): 1,
#          ('A', 'AB'): 1,
#          ('O', 'AB'): 1})
# optPaircount = pd.DataFrame.from_dict(p, orient='index').reset_index()

# optPaircount

,index,0
0,"(A, O)",18
1,"(O, A)",17
2,"(O, O)",22
3,"(B, A)",4
4,"(A, B)",3
5,"(O, B)",3
6,"(B, O)",3
7,"(A, A)",7
8,"(AB, A)",1
9,"(AB, AB)",1


In [27]:
# #these are all the cities of the pairs from the original data

# allcityv2 = []
# for i in range(len(pairs)):
#     if pairs.iloc[i]["type"] == "pair":
#         allcity = ((pairs.iloc[i]["Location"]))
#         allcityv2.append(allcity)
        
  
# Counter(allcityv2)

# o = Counter({'Moscow': 10,
#          'Frankfurt': 6,
#          'Vienna': 4,
#          'Milan': 3,
#          'Madrid': 3,
#          'Düsseldorf': 7,
#          'Lisbon': 5,
#          'Amsterdam': 6,
#          'Oslo': 4,
#          'Zürich': 4,
#          'Copenhagen': 7,
#          'Istanbul': 9,
#          'London': 6,
#          'Palma de Mallorca': 6,
#          'Manchester': 6,
#          'Dublin': 5,
#          'Rome': 3,
#          'Paris': 2})
# allcityCount = pd.DataFrame.from_dict(o, orient='index').reset_index()
# allcityCount

,index,0
0,Moscow,10
1,Frankfurt,6
2,Vienna,4
3,Milan,3
4,Madrid,3
5,Düsseldorf,7
6,Lisbon,5
7,Amsterdam,6
8,Oslo,4
9,Zürich,4


In [28]:
# #These are the cities of the pairs from the optimal solution

# cityopt = []
# for i in range(len(pairedrecdf)):
#     for j in range(len(pairs)):
#         if pairedrecdf.iloc[i][0] == pairs.iloc[j]["ID"]:
#             locator = (pairs.iloc[j]["Location"])
#             cityopt.append(locator)
# Counter(cityopt)



# d = Counter({'Frankfurt': 6,
#          'Vienna': 3,
#          'Milan': 3,
#          'Madrid': 3,
#          'Düsseldorf': 7,
#          'Moscow': 5,
#          'Lisbon': 5,
#          'Amsterdam': 5,
#          'Oslo': 3,
#          'Zürich': 4,
#          'Copenhagen': 4,
#          'Istanbul': 8,
#          'London': 5,
#          'Palma de Mallorca': 5,
#          'Manchester': 6,
#          'Dublin': 4,
#          'Rome': 3,
#          'Paris': 2})
# cityoptCount = pd.DataFrame.from_dict(d, orient='index').reset_index()
# cityoptCount

,index,0
0,Frankfurt,6
1,Vienna,3
2,Milan,3
3,Madrid,3
4,Düsseldorf,7
5,Moscow,5
6,Lisbon,5
7,Amsterdam,5
8,Oslo,3
9,Zürich,4


## Question 2.1

Re-running the generation of C here since we overwrote it for question 1.5

In [45]:
def computePaths(arcs):
    paths = [[arc] for arc in arcs]
    join = True
    while(join):
        join = False
        for i,p1 in enumerate(paths):
            if join == True:
                break
            for j,p2 in enumerate(paths):
                if i<j:
                    if p1[-1][1] == p2[0][0]:
                        p2 = paths.pop(j)
                        paths[i] = paths[i]+p2
                        join = True
                    elif p1[0][0] == p2[-1][1]:
                        p2 = paths.pop(j)
                        paths[i] = p2 + paths[i]
                        join = True
    return paths

In [46]:
#creating a for loop to find the longest path/chain and its length 
optPaths = computePaths(optSolution)
maxlength = 0
for path in optPaths:
    if len(path) > maxlength:
        maxlength = len(path)
        longestPath = path

print(longestPath)
print(maxlength)

[(186, 111), (111, 46), (46, 94), (94, 159), (159, 104), (104, 145), (145, 103), (103, 167), (167, 49)]
9


## Question 2.2

In [47]:
#here we just copied the code from above, but renamed the model to model2 to be able to easily compare it to the 
#previous model
model2 = gp.Model()
x = model2.addVars(C, vtype = gp.GRB.BINARY, name = " ")
model2.setObjective(gp.quicksum(x[match] for match in C),gp.GRB.MAXIMIZE)
model2.addConstrs(gp.quicksum(x[i,j] for j in (dfC[dfC[0]==i][1])) <= 1 for i in (potential_donors))
model2.addConstrs(gp.quicksum(x[i,j] for i in (dfC[dfC[1]==j][0])) <= 1 for j in (potential_rec))
model2.addConstrs(gp.quicksum(x[i,j] for j in (dfC[dfC[0]==i][1])) == 
                  gp.quicksum(x[k,i] for k in (dfC[dfC[1]==i][0])) for i in potential_pairs)
model2.optimize()
model2.printAttr("X")

#defining separately the optimal solution for model 2
optSolution2 = [k for k, v in x.items() if v.X > 0] 
optPaths = computePaths(optSolution2)

#we define the for loop for finding paths that are longer than 3
longerPaths = []
for path in optPaths:
    if len(path) > 3:
        longerPaths.append(path)

#creating a while loop, which keeps running while the longerPaths list is 
while len(longerPaths) > 0:
    for path in longerPaths:
        model2.addConstrs(((x[path[i]]+x[path[i+1]]+x[path[i+2]]+x[path[i+3]]) <= 3) for i in (range(len(path)-3)))
    model2.optimize()
    optSolution2 = [k for k, v in x.items() if v.X > 0] 
    optPaths = computePaths(optSolution2)
    longerPaths = []
    for path in optPaths:
        if len(path) > 3:
            longerPaths.append(path)

    

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 369 rows, 896 columns and 2932 nonzeros
Model fingerprint: 0x34755998
Variable types: 0 continuous, 896 integer (896 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 45.0000000
Presolve removed 192 rows and 251 columns
Presolve time: 0.02s
Presolved: 177 rows, 645 columns, 2057 nonzeros
Found heuristic solution: objective 75.0000000
Variable types: 0 continuous, 645 integer (645 binary)

Root relaxation: objective 9.000000e+01, 167 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      90.0000000   90.00000  0.00

Optimize a model with 438 rows, 896 columns and 3208 nonzeros
Model fingerprint: 0xbf7b2817
Variable types: 0 continuous, 896 integer (896 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R426 by 1.000000000

Found heuristic solution: objective 45.0000000
Presolve removed 207 rows and 251 columns
Presolve time: 0.02s
Presolved: 231 rows, 645 columns, 2263 nonzeros
Found heuristic solution: objective 75.0000000
Variable types: 0 continuous, 645 integer (645 binary)

Root relaxation: objective 9.000000e+01, 168 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      90.0000000   90

     0     0          -    0        90.00000   90.00000  0.00%     -    0s

Explored 1 nodes (299 simplex iterations) in 0.06 seconds (0.01 work units)
Thread count was 8 (of 8 available processors)

Solution count 3: 90 75 45 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.000000000000e+01, best bound 9.000000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 526 rows, 896 columns and 3560 nonzeros
Model fingerprint: 0x8cbaadba
Variable types: 0 continuous, 896 integer (896 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R507 by 1.000000000

Found heuristic solution: objective 45.0000000
Presolve removed 211 

  RHS range        [1e+00, 3e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R589 by 1.000000000

Found heuristic solution: objective 45.0000000
Presolve removed 212 rows and 251 columns
Presolve time: 0.02s
Presolved: 393 rows, 645 columns, 2880 nonzeros
Found heuristic solution: objective 75.0000000
Variable types: 0 continuous, 645 integer (645 binary)

Root relaxation: objective 9.000000e+01, 204 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                      90.0000000  701.00000   679%     -    0s
     0     0          -    0        90.00000   90.00000  0.00%     -    0s

Explored 1 nodes (309 simplex iterations) in 0.05 seconds (0.01 work units)
Thread count was 8 (of 8 available processors)

Solution count 3: 90 75 45 

Optimal solution f


Explored 1 nodes (217 simplex iterations) in 0.05 seconds (0.01 work units)
Thread count was 8 (of 8 available processors)

Solution count 3: 90 75 45 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.000000000000e+01, best bound 9.000000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 681 rows, 896 columns and 4180 nonzeros
Model fingerprint: 0x2b616de9
Variable types: 0 continuous, 896 integer (896 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R663 by 1.000000000

Found heuristic solution: objective 45.0000000
Presolve removed 215 rows and 251 columns
Presolve time: 0.03s
Presolved: 466 rows, 645 columns,

...

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                      90.0000000  701.00000   679%     -    0s
     0     0          -    0        90.00000   90.00000  0.00%     -    0s

Explored 1 nodes (638 simplex iterations) in 0.13 seconds (0.08 work units)
Thread count was 8 (of 8 available processors)

Solution count 3: 90 75 45 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.000000000000e+01, best bound 9.000000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3031 rows, 896 columns and 13580 nonzeros
Model fingerprint: 0xd242b95c
Variable types: 0 continuous, 896 integer (896 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]

MIP start from previous solve did not produce a new inc

Thread count was 8 (of 8 available processors)

Solution count 3: 90 75 45 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.000000000000e+01, best bound 9.000000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3073 rows, 896 columns and 13748 nonzeros
Model fingerprint: 0xac45fedb
Variable types: 0 continuous, 896 integer (896 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R3063 by 1.000000000

Found heuristic solution: objective 45.0000000
Presolve removed 254 rows and 251 columns
Presolve time: 0.06s
Presolved: 2819 rows, 645 columns, 12338 nonzeros
Found heuristic solution: objective 75.0000000
Variable t

Best objective 9.000000000000e+01, best bound 9.000000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3110 rows, 896 columns and 13896 nonzeros
Model fingerprint: 0xddbf76f2
Variable types: 0 continuous, 896 integer (896 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R3103 by 1.000000000

Found heuristic solution: objective 45.0000000
Presolve removed 254 rows and 251 columns
Presolve time: 0.06s
Presolved: 2856 rows, 645 columns, 12478 nonzeros
Found heuristic solution: objective 75.0000000
Variable types: 0 continuous, 645 integer (645 binary)

Root relaxation: objective 9.000000e+01, 268 iterations, 0.02 seconds (0.01

Best objective 9.000000000000e+01, best bound 9.000000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3149 rows, 896 columns and 14052 nonzeros
Model fingerprint: 0x61c78155
Variable types: 0 continuous, 896 integer (896 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R3143 by 1.000000000

Found heuristic solution: objective 45.0000000
Presolve removed 255 rows and 251 columns
Presolve time: 0.06s
Presolved: 2894 rows, 645 columns, 12621 nonzeros
Found heuristic solution: objective 75.0000000
Variable types: 0 continuous, 645 integer (645 binary)

Root relaxation: objective 9.000000e+01, 277 iterations, 0.02 seconds (0.01

  RHS range        [1e+00, 3e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R3179 by 1.000000000

Found heuristic solution: objective 45.0000000
Presolve removed 255 rows and 251 columns
Presolve time: 0.06s
Presolved: 2932 rows, 645 columns, 12769 nonzeros
Found heuristic solution: objective 75.0000000
Variable types: 0 continuous, 645 integer (645 binary)

Root relaxation: objective 9.000000e+01, 250 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                      90.0000000  701.00000   679%     -    0s
     0     0          -    0        90.00000   90.00000  0.00%     -    0s

Explored 1 nodes (435 simplex iterations) in 0.12 seconds (0.07 work units)
Thread count was 8 (of 8 available processors)

Solution count 3: 90 75 45 

Optimal solutio

In [50]:
optPaths

[[(133, 4), (4, 30), (30, 133)],
 [(41, 8), (8, 194), (194, 70)],
 [(10, 33), (33, 10)],
 [(14, 164), (164, 118)],
 [(175, 19), (19, 182)],
 [(20, 93), (93, 20)],
 [(29, 29)],
 [(31, 103), (103, 167), (167, 49)],
 [(32, 32)],
 [(34, 34)],
 [(35, 102), (102, 35)],
 [(36, 146)],
 [(54, 40), (40, 180), (180, 54)],
 [(148, 42), (42, 82), (82, 148)],
 [(131, 46), (46, 94), (94, 48)],
 [(55, 161), (161, 171), (171, 109)],
 [(58, 58)],
 [(162, 60), (60, 145), (145, 162)],
 [(61, 89), (89, 196), (196, 65)],
 [(63, 159), (159, 76), (76, 79)],
 [(64, 84), (84, 197), (197, 38)],
 [(67, 115), (115, 101)],
 [(68, 113), (113, 147), (147, 28)],
 [(73, 73)],
 [(85, 110), (110, 85)],
 [(105, 87), (87, 151)],
 [(136, 91), (91, 199), (199, 52)],
 [(95, 99), (99, 152), (152, 112)],
 [(155, 191), (191, 97), (97, 11)],
 [(104, 104)],
 [(108, 108)],
 [(186, 111), (111, 47)],
 [(114, 114)],
 [(116, 116)],
 [(117, 117)],
 [(123, 119), (119, 143), (143, 123)],
 [(120, 120)],
 [(121, 132), (132, 121)],
 [(122, 1

## Question 2.3

In [51]:
print(len(optSolution2))
model.printAttr("NumConstrs")
model2.printAttr("NumConstrs")

90

  NumConstrs 
------------
         369 

  NumConstrs 
------------
        3222 


In [ ]:
print (model2.display())